In [2]:
from pathlib import Path
import pickle
import zipfile

import numpy as np
import pandas as pd

import rdflib
from rdflib import Graph
from rdflib.namespace import BRICK

In [3]:
dataset_dir = '../../datasets/bts_site_b_train/'

dataset_zip = 'train.zip'
dataset_path = Path(dataset_dir) / dataset_zip

mapping_csv = 'mapper_TrainOnly.csv'
mapping_path = Path(dataset_dir) / mapping_csv

building_ttl = 'Site_B.ttl'
building_model = Path(dataset_dir) / building_ttl

brick_ttl = 'Brick_v1.2.1.ttl'
brick_schema = Path(dataset_dir) / brick_ttl

In [4]:
# Load the mapping file
mapping_df = pd.read_csv(mapping_path, index_col=0)

# Building B only
# mapping_df = mapping_df[mapping_df['Building'] == 'B']

# Ignore streams not saved to file
mapping_df = mapping_df[mapping_df['Filename'].str.contains('FILE NOT SAVED') == False]

mapping_df.head()

,Building,StreamID,Filename,strBrickLabel
0,A,9ba955fa_5960_4c9b_b73a_10156da7d083,trainAll_0.pkl,Operating_Mode_Status
2,A,8fd6e75b_88bc_4992_b420_77389969b3c4,trainAll_1.pkl,Mode_Command
3,A,8db6eaa9_bd6c_4f7e_aed0_a47e4e192a6c,trainAll_2.pkl,Active_Power_Sensor
4,A,b2338dec_110a_45cc_8358_1171aaef2c45,trainAll_3.pkl,System_Status
5,A,ec5ff874_0af2_49d8_a6a0_21ea3d077dc8,trainAll_4.pkl,Maintenance_Mode_Command


In [6]:
streams = []

with zipfile.ZipFile(dataset_path, "r") as f:
    for name in f.namelist():
        if not name.endswith('.pkl'):
            continue

        pkl_data = f.read(name)
        data = pickle.loads(pkl_data)

        filename = name.split('/')[-1]

        building = mapping_df.loc[mapping_df['Filename'] == filename, 'Building'].iloc[0]
        stream_id = mapping_df.loc[mapping_df['Filename'] == filename, 'StreamID'].iloc[0]
        brick_class_csv = mapping_df.loc[mapping_df['Filename'] == filename, 'strBrickLabel'].iloc[0]
        brick_class_pkl = data['y']
        data_points = len(data['t'])
        min_time = min(data['t'])
        max_time = max(data['t'])
        min_value = min(data['v'])
        mean_value = np.mean(data['v'])
        median_value = np.median(data['v'])
        max_value = max(data['v'])

        # print('Building:', building, 'StreamID:', stream_id, 'Brick Class:', brick_class, 'Min Time:', min_time, 'Max Time:', max_time, 'Min Value:', min_value, 'Max Value:', max_value)
        # break

        streams.append({
            'filename': filename,
            'building': building,
            'stream_id': stream_id,
            'brick_class_pkl': brick_class_pkl,
            'brick_class_csv': brick_class_csv,
            'data_points': data_points,
            'min_time': min_time,
            'max_time': max_time,
            'min_value': min_value,
            'max_value': max_value,
            'mean_value': mean_value,
            'median_value': median_value,
        })

/var/folders/_9/s1xwzrv537n2xj30q_b98s380000gn/T/ipykernel_51333/1134885109.py:9: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  data = pickle.loads(pkl_data)


In [7]:
summary_df = pd.DataFrame(streams)
summary_df.head()

,filename,building,stream_id,brick_class_pkl,brick_class_csv,data_points,min_time,max_time,min_value,max_value,mean_value,median_value
0,trainAll_2183.pkl,A,980d7c24_3626_47f0_9430_582c827ee507,Alarm,Alarm,12939,2021-01-01 00:09:56.621,2021-03-31 22:51:21.721,0.0,0.0,0.000000,0.0
1,trainAll_5834.pkl,A,3b9ab701_ec53_491f_a943_d82b3e90f130,Mode_Status,Mode_Status,12902,2021-01-01 00:02:30.508,2021-03-31 22:54:07.521,0.0,1.0,0.203069,0.0
2,trainAll_6483.pkl,A,22fd94c6_f39f_49b1_b3af_36b4dffd90a9,Fan_Status,Fan_Status,12904,2021-01-01 00:02:45.031,2021-03-31 22:54:18.431,0.0,1.0,0.288205,0.0
3,trainAll_7945.pkl,A,1e352003_b4c7_43f0_8f8b_099d44897fe6,Status,Status,12893,2021-01-01 00:03:01.414,2021-03-31 22:54:35.893,0.0,1.0,0.581478,1.0
4,trainAll_4294.pkl,A,7260e1bc_3b09_4880_8c68_57b50b4f0f93,Alarm,Alarm,12922,2021-01-01 00:00:18.577,2021-03-31 22:51:45.823,0.0,0.0,0.000000,0.0


In [8]:
g = Graph().parse(building_model, format='turtle')

In [9]:
def get_brick_class(stream_id, graph):
    predicate = rdflib.term.URIRef('http://senaps.io/schema/1.0/senaps#stream_id')
    object = rdflib.term.Literal(stream_id)
    subject = g.value(predicate=predicate, object=object, any=True)

    predicate = rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type')
    brick_class = graph.value(subject, predicate)

    if brick_class is None:
        return None
    
    return brick_class.replace('https://brickschema.org/schema/Brick#', '')

In [10]:
summary_df['brick_class_ttl'] = summary_df['stream_id'].apply(get_brick_class, args=(g,))
summary_df.head()

,filename,building,stream_id,brick_class_pkl,brick_class_csv,data_points,min_time,max_time,min_value,max_value,mean_value,median_value,brick_class_ttl
0,trainAll_2183.pkl,A,980d7c24_3626_47f0_9430_582c827ee507,Alarm,Alarm,12939,2021-01-01 00:09:56.621,2021-03-31 22:51:21.721,0.0,0.0,0.000000,0.0,None
1,trainAll_5834.pkl,A,3b9ab701_ec53_491f_a943_d82b3e90f130,Mode_Status,Mode_Status,12902,2021-01-01 00:02:30.508,2021-03-31 22:54:07.521,0.0,1.0,0.203069,0.0,None
2,trainAll_6483.pkl,A,22fd94c6_f39f_49b1_b3af_36b4dffd90a9,Fan_Status,Fan_Status,12904,2021-01-01 00:02:45.031,2021-03-31 22:54:18.431,0.0,1.0,0.288205,0.0,None
3,trainAll_7945.pkl,A,1e352003_b4c7_43f0_8f8b_099d44897fe6,Status,Status,12893,2021-01-01 00:03:01.414,2021-03-31 22:54:35.893,0.0,1.0,0.581478,1.0,None
4,trainAll_4294.pkl,A,7260e1bc_3b09_4880_8c68_57b50b4f0f93,Alarm,Alarm,12922,2021-01-01 00:00:18.577,2021-03-31 22:51:45.823,0.0,0.0,0.000000,0.0,None


In [11]:
def get_relative_of_stream_source(stream_id, tag, g):
    # get the source of the stream
    stream_id = rdflib.term.Literal(stream_id)
    stream_pred = rdflib.term.URIRef('http://senaps.io/schema/1.0/senaps#stream_id')
    stream_source = g.value(predicate=stream_pred, object=stream_id, any=True)
    if stream_source is None:
        return None
    
    # get the relative of the stream source
    stream_source_pred = BRICK[tag]
    stream_source_object = g.value(subject=stream_source, predicate=stream_source_pred, any=True)

    if stream_source_object is None:
        return None
    
    # get the class of the relative
    stream_source_class_pred = rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type')
    stream_source_class = g.value(subject=stream_source_object, predicate=stream_source_class_pred, any=True)
    
    if stream_source_class is None:
        return None

    return stream_source_class.replace('https://brickschema.org/schema/Brick#', '')

In [12]:
summary_df['isPointOf'] = summary_df['stream_id'].apply(get_relative_of_stream_source, args=('isPointOf', g))
summary_df['hasLocation'] = summary_df['stream_id'].apply(get_relative_of_stream_source, args=('hasLocation', g))
summary_df.head()

,filename,building,stream_id,brick_class_pkl,brick_class_csv,data_points,min_time,max_time,min_value,max_value,mean_value,median_value,brick_class_ttl,isPointOf,hasLocation
0,trainAll_2183.pkl,A,980d7c24_3626_47f0_9430_582c827ee507,Alarm,Alarm,12939,2021-01-01 00:09:56.621,2021-03-31 22:51:21.721,0.0,0.0,0.000000,0.0,None,None,None
1,trainAll_5834.pkl,A,3b9ab701_ec53_491f_a943_d82b3e90f130,Mode_Status,Mode_Status,12902,2021-01-01 00:02:30.508,2021-03-31 22:54:07.521,0.0,1.0,0.203069,0.0,None,None,None
2,trainAll_6483.pkl,A,22fd94c6_f39f_49b1_b3af_36b4dffd90a9,Fan_Status,Fan_Status,12904,2021-01-01 00:02:45.031,2021-03-31 22:54:18.431,0.0,1.0,0.288205,0.0,None,None,None
3,trainAll_7945.pkl,A,1e352003_b4c7_43f0_8f8b_099d44897fe6,Status,Status,12893,2021-01-01 00:03:01.414,2021-03-31 22:54:35.893,0.0,1.0,0.581478,1.0,None,None,None
4,trainAll_4294.pkl,A,7260e1bc_3b09_4880_8c68_57b50b4f0f93,Alarm,Alarm,12922,2021-01-01 00:00:18.577,2021-03-31 22:51:45.823,0.0,0.0,0.000000,0.0,None,None,None


In [13]:
b = Graph().parse(brick_schema)

In [14]:
def get_brick_definition(brick_class, graph):
    if brick_class is None:
        return None
    subject = BRICK[brick_class]
    predicate = rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#definition')
    definition = graph.value(subject, predicate)
    return definition

In [15]:
summary_df['brick_definition'] = summary_df['brick_class_ttl'].apply(get_brick_definition, args=(b,))
summary_df.head()

/Users/tarney/code/unsw/comp9900/thecapstoners/.venv/lib/python3.12/site-packages/pandas/core/apply.py:1496: UserWarning: Code: b1a78aa6_d50d_4c0a_920e_5ff7019ab663_Sensor is not defined in namespace BRICK
  return func(x, *self.args, **self.kwargs)
/Users/tarney/code/unsw/comp9900/thecapstoners/.venv/lib/python3.12/site-packages/pandas/core/apply.py:1496: UserWarning: Code: Electrical_Energy_Sensor is not defined in namespace BRICK
  return func(x, *self.args, **self.kwargs)


,filename,building,stream_id,brick_class_pkl,brick_class_csv,data_points,min_time,max_time,min_value,max_value,mean_value,median_value,brick_class_ttl,isPointOf,hasLocation,brick_definition
0,trainAll_2183.pkl,A,980d7c24_3626_47f0_9430_582c827ee507,Alarm,Alarm,12939,2021-01-01 00:09:56.621,2021-03-31 22:51:21.721,0.0,0.0,0.000000,0.0,None,None,None,None
1,trainAll_5834.pkl,A,3b9ab701_ec53_491f_a943_d82b3e90f130,Mode_Status,Mode_Status,12902,2021-01-01 00:02:30.508,2021-03-31 22:54:07.521,0.0,1.0,0.203069,0.0,None,None,None,None
2,trainAll_6483.pkl,A,22fd94c6_f39f_49b1_b3af_36b4dffd90a9,Fan_Status,Fan_Status,12904,2021-01-01 00:02:45.031,2021-03-31 22:54:18.431,0.0,1.0,0.288205,0.0,None,None,None,None
3,trainAll_7945.pkl,A,1e352003_b4c7_43f0_8f8b_099d44897fe6,Status,Status,12893,2021-01-01 00:03:01.414,2021-03-31 22:54:35.893,0.0,1.0,0.581478,1.0,None,None,None,None
4,trainAll_4294.pkl,A,7260e1bc_3b09_4880_8c68_57b50b4f0f93,Alarm,Alarm,12922,2021-01-01 00:00:18.577,2021-03-31 22:51:45.823,0.0,0.0,0.000000,0.0,None,None,None,None


In [18]:
summary_df.sort_values(by=['building', 'brick_class_pkl', 'brick_class_ttl', 'isPointOf','filename'], inplace=True)
summary_df.insert(5, 'brick_class_ttl', summary_df.pop('brick_class_ttl'))
summary_df.insert(6, 'isPointOf', summary_df.pop('isPointOf'))
summary_df.insert(7, 'hasLocation', summary_df.pop('hasLocation'))
summary_df.head()

,filename,building,stream_id,brick_class_pkl,brick_class_csv,brick_class_ttl,isPointOf,hasLocation,data_points,min_time,max_time,min_value,max_value,mean_value,median_value,brick_definition
3078,trainAll_1010.pkl,A,985db5d1_04e4_41cc_b954_68a6bc41f309,Active_Power_Sensor,Active_Power_Sensor,None,None,None,10648,2021-01-01 00:13:50.687,2021-03-31 22:55:08.042,0.0,0.5265,0.432480,0.4634,None
1444,trainAll_1062.pkl,A,b95f79bb_0df0_4395_a3aa_f49beda0d401,Active_Power_Sensor,Active_Power_Sensor,None,None,None,7101,2021-01-01 00:14:07.051,2021-03-31 22:55:21.914,0.0,6.3675,1.273096,1.1124,None
360,trainAll_1064.pkl,A,49cc1fa6_431c_40e0_b151_1adf32169dcd,Active_Power_Sensor,Active_Power_Sensor,None,None,None,10671,2021-01-01 00:13:50.408,2021-03-31 22:55:07.875,0.0,1.1900,0.714675,0.6370,None
205,trainAll_1071.pkl,A,4c15ddb0_d74e_4938_a774_99c92be49f23,Active_Power_Sensor,Active_Power_Sensor,None,None,None,11659,2021-01-01 00:03:26.957,2021-03-31 22:55:02.216,0.0,2.1414,1.460795,1.8466,None
7439,trainAll_1085.pkl,A,a7918557_0c82_4341_ae95_52024debeedb,Active_Power_Sensor,Active_Power_Sensor,None,None,None,12912,2021-01-01 00:02:07.587,2021-03-31 22:53:47.224,0.0,36.5542,19.191632,18.6096,None


In [19]:
summary_df.to_csv('bts_site_b_train_summary.csv', index=False)